# Acquire

In [1]:
from env import host, user, password
import acquire
import prepare
import wrangle

In [2]:
df = acquire.get_zillow()

In [3]:
df.head()

,calculatedfinishedsquarefeet,bedroomcnt,bathroomcnt,taxvaluedollarcnt,taxamount
0,3100.0,4.0,3.5,1023282.0,11013.72
1,1465.0,2.0,1.0,464000.0,5672.48
2,1243.0,3.0,2.0,564778.0,6488.30
3,2376.0,4.0,3.0,145143.0,1777.51
4,1312.0,3.0,3.0,119407.0,1533.89


In [4]:
#calculate tax rate
#df['tax_rate'] =df['taxamount']/  df['taxvaluedollarcnt']

In [5]:
#df.head()

# Prepare

In [6]:
zillow= prepare.prep_zillow(df)

In [7]:
zillow.head()

,calculatedfinishedsquarefeet,bedroomcnt,bathroomcnt,taxvaluedollarcnt,taxamount,tax_rate
0,3100.0,4.0,3.5,1023282.0,11013.72,0.010763
1,1465.0,2.0,1.0,464000.0,5672.48,0.012225
2,1243.0,3.0,2.0,564778.0,6488.30,0.011488
3,2376.0,4.0,3.0,145143.0,1777.51,0.012247
4,1312.0,3.0,3.0,119407.0,1533.89,0.012846


In [8]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76940 entries, 0 to 77612
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   calculatedfinishedsquarefeet  76940 non-null  float64
 1   bedroomcnt                    76940 non-null  float64
 2   bathroomcnt                   76940 non-null  float64
 3   taxvaluedollarcnt             76940 non-null  float64
 4   taxamount                     76940 non-null  float64
 5   tax_rate                      76940 non-null  float64
dtypes: float64(6)
memory usage: 4.1 MB


In [16]:
#split 
X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.train_validate_test(zillow, 'taxvaluedollarcnt')

X_train -> (43086, 5)
X_validate -> (18466, 5)
X_test -> (15388, 5)


In [22]:
#take a look at X_train
X_train.head()

,calculatedfinishedsquarefeet,bedroomcnt,bathroomcnt,taxamount,tax_rate
71395,1047.0,3.0,2.0,2419.97,0.012919
61620,953.0,2.0,2.0,3294.48,0.010557
73519,1302.0,3.0,2.0,5412.36,0.010268
76582,2612.0,5.0,4.0,8188.78,0.012356
50090,1865.0,3.0,3.0,13324.52,0.016855


In [18]:
#add numeric columns to be scaled (except y_train (taxvaluedollarcnt))
numeric_cols = ['calculatedfinishedsquarefeet', 'bedroomcnt', 'bathroomcnt', 'taxamount', 'tax_rate']

In [20]:
#scale data
X_train_scaled, X_validate_scaled, X_test_scaled = wrangle.min_max_scale(X_train, X_validate, X_test, numeric_cols)

In [23]:
#now take a look at X_train after being scaled (minmaxscaler)
X_train_scaled.head()

,calculatedfinishedsquarefeet,bedroomcnt,bathroomcnt,taxamount,tax_rate
71395,0.025879,0.1875,0.181818,0.008248,0.015709
61620,0.023232,0.1250,0.181818,0.011254,0.012816
73519,0.033059,0.1875,0.181818,0.018532,0.012463
76582,0.069948,0.3125,0.363636,0.028074,0.015020
50090,0.048913,0.1875,0.272727,0.045724,0.020531


# Explore

In [14]:
X_t.taxvaluedollarcnt.value_counts()

400000.0    54
600000.0    48
350000.0    47
450000.0    45
500000.0    43
            ..
263024.0     1
60886.0      1
217006.0     1
302939.0     1
98304.0      1
Name: taxvaluedollarcnt, Length: 50809, dtype: int64